<a href="https://colab.research.google.com/github/Gregory-lab-eng/python_Vistula/blob/main/Profile_picture_validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install gradio_client

In [ ]:
from transformers import BlipForQuestionAnswering, BlipProcessor, AutoProcessor, AutoModelForImageClassification
from PIL import Image
import gradio as gr
import requests

In [ ]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "image-classification",
    model="Falconsai/nsfw_image_detection"
)

def check_image_safety(img):
    if not isinstance(img, Image.Image):
        img = Image.fromarray(img)

    result = pipe(img)[0]
    label = result["label"].upper()

    return ("YES" if label != "SAFE" else "NO"), result


Z wielu ofierowanych modeli ten się okazał najdokladnejszym przy klassifikacji twarzy osób.

Bardzo ważnie było wybrać precyzyjne pytannie dla sprawdzania zdjęcia, perwsza versja była "Is it a photo of a person?" - co spowodowało że przy dodaniu róznych części ciała model odpowiadał "tak".
Dalej pytanie było doprecyzowane i sprawdzone, na razie wszystkie testy pokazują że program działa dokładnie.

Sprawdzamy wybrany model na dokładność:

In [ ]:

def save_and_process(img):
    safety_check = check_image_safety(img)
    print(safety_check[0])
    if safety_check[0] == "YES":
        image = Image.fromarray(img)
        image.save("uploaded_image.png")
        img_url = '/content/uploaded_image.png'

        question = "Is it a person with a face?"
        inputs = processor(img_url, question, return_tensors="pt")
        outputs = model.generate(**inputs)
        model_answer = processor.decode(outputs[0], skip_special_tokens=True)

        if model_answer == "yes":
            reply = "This is a photo of a person you can proceed"
        else:
            reply = "Please upload a photo of a person where face is visible"
        return reply
    else:
        return "This photo is not complying with the safety policy, please upload different one"


sprawdzanie modelu na różnych typach person


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_photo(img):
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:
img = mpimg.imread('/content/photo-1568782517100-09bf22d88c2d.avif')
show_photo(img)
save_and_process(img)

In [ ]:
img = mpimg.imread('/content/photo-1518882570151-157128e78fa1.avif')
show_photo(img)
save_and_process(img)

sprawdzanie na części ciała:

In [ ]:
img = mpimg.imread('/content/photo-1518601794912-1af91724e528.avif')
show_photo(img)
save_and_process(img)

### Widać że model w 100% rozpoznaje różne typy, więc stwarzamy endpoint dla korzystania rożnych deweloperow przez API, korzystając z biblioteki gradio:

In [ ]:
demo = gr.Interface(
    fn=save_and_process,
    inputs=gr.Image(type="numpy"),
    outputs="text"
)

demo.launch()